# Integrate external test providers

Register a custom test provider with the Validmind Developer Framework to run your own tests.

This notebook loads two of sample test providers and registers them with the framework which enables you to run a template that utilizes the custom tests made available by the providers.


## Contents
- [About ValidMind](#toc1_)    
  - [Before you begin](#toc1_1_)    
  - [New to ValidMind?](#toc1_2_)    
  - [Key concepts](#toc1_3_)    
- [Install the client library](#toc2_)    
- [Update the customer churn template template](#toc3_)    
- [Initialize the client library](#toc4_)    
  - [Preview the documentation template and validate that it contains the new test blocks](#toc4_1_)    
  - [Register external test providers](#toc4_2_)    
  - [Verify that `preview_template()` now loads the tests from the test providers](#toc4_3_)    
  - [Running the template](#toc4_4_)    
- [Next steps](#toc5_)    
  - [Work with your model documentation](#toc5_1_)    
  - [Discover more learning resources](#toc5_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

<a id='toc1_'></a>

## About ValidMind

ValidMind is a platform for managing model risk, including risk associated with AI and statistical models. You use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

<a id='toc1_1_'></a>

### Before you begin

This notebook assumes you have basic familiarity with Python, including an understanding of how functions work. If you are new to Python, you can still run the notebook but we recommend further familiarizing yourself with the language. 

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

<a id='toc1_2_'></a>

### New to ValidMind?

If you haven't already seen our [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html), we recommend you explore the available resources for developers at some point. There, you can learn more about documenting models, find code samples, or read our developer reference.

::: {.callout-tip}

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)

:::

<a id='toc1_3_'></a>

### Key concepts

**Model documentation**: A structured and detailed record pertaining to a model, encompassing key components such as its underlying assumptions, methodologies, data sources, inputs, performance metrics, evaluations, limitations, and intended uses. It serves to ensure transparency, adherence to regulatory requirements, and a clear understanding of potential risks associated with the model’s application.

**Documentation template**: Functions as a test suite and lays out the structure of model documentation, segmented into various sections and sub-sections. Documentation templates define the structure of your model documentation, specifying the tests that should be run, and how the results should be displayed.

**Tests**: A function contained in the ValidMind Developer Framework, designed to run a specific quantitative test on the dataset or model. Tests are the building blocks of ValidMind, used to evaluate and document models and datasets, and can be run individually or as part of a suite defined by your model documentation template.

**Metrics**: A subset of tests that do not have thresholds. In the context of this notebook, metrics and tests can be thought of as interchangeable concepts.

**Custom metrics**: Custom metrics are functions that you define to evaluate your model or dataset. These functions can be registered with ValidMind to be used in the platform.

**Inputs**: Objects to be evaluated and documented in the ValidMind framework. They can be any of the following:

  - **model**: A single model that has been initialized in ValidMind with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model).
  - **dataset**: Single dataset that has been initialized in ValidMind with [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset).
  - **models**: A list of ValidMind models - usually this is used when you want to compare multiple models in your custom metric.
  - **datasets**: A list of ValidMind datasets - usually this is used when you want to compare multiple datasets in your custom metric. See this [example](https://docs.validmind.ai/notebooks/how_to/run_tests_that_require_multiple_datasets.html) for more information.

**Parameters**: Additional arguments that can be passed when running a ValidMind test, used to pass additional information to a metric, customize its behavior, or provide additional context.

**Outputs**: Custom metrics can return elements like tables or plots. Tables may be a list of dictionaries (each representing a row) or a pandas DataFrame. Plots may be matplotlib or plotly figures.

**Test suites**: Collections of tests designed to run together to automate and generate model documentation end-to-end for specific use-cases.

Example: the [`classifier_full_suite`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html#ClassifierFullSuite) test suite runs tests from the [`tabular_dataset`](https://docs.validmind.ai/validmind/validmind/test_suites/tabular_datasets.html) and [`classifier`](https://docs.validmind.ai/validmind/validmind/test_suites/classifier.html) test suites to fully document the data and model sections for binary classification model use-cases.


<a id='toc2_'></a>

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [ ]:
%pip install -q validmind

<a id='toc3_'></a>

## Update the customer churn template template

First, let's edit the **Binary classification** template and register test blocks for the demo test providers we will implement below.

- Go to **Settings > Templates** and click on the **Binary classification** template. Let's add a new top level section called `test_providers_demo` with some test driven content blocks like below:

```yaml
- id: test_providers_demo
  title: Test providers demo
  contents:
    - content_type: metric
      content_id: my_local_provider.tests.MyCustomTest
    - content_type: metric
      content_id: my_inline_provider.tests.MyCustomTest
```

- Click on **Prepare new version**, provide some version notes and click con **Save new version** to save a new version of this template
- Now we need to swap our model documentation to this new version of the template. Follow the steps on this guide to swap the template of our customer churn model: https://docs.validmind.ai/guide/swap-documentation-templates.html

In the sections below we provide more context on how these `content_id`s above get mapped to the actual test providers and tests.


<a id='toc4_'></a>

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details, making sure to select the `Attrition/Churn Management` use case, and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:


In [ ]:
# Replace with your code snippet

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="...",
)

<a id='toc4_1_'></a>

### Preview the documentation template and validate that it contains the new test blocks

We should see two custom content blocks in the template whose IDs are under the namespaces registered below (`my_inline_provider` and `my_local_provider`).

::: {.callout-tip}

`preview_template()` will show an error when loading the new tests since we haven't registered our new test providers yet. This is expected.

:::


In [ ]:
vm.preview_template()

<a id='toc4_2_'></a>

### Register external test providers

We will now instantiate and register test provider classes that include the tests we included in the template.

We will build an inline test provider that allows creating tests directly in the notebook and a local filesystem test provider that allows loading tests from a local folder.

For the local filesystem provider, we just need to specify the root folder under which the provider class will look for tests. For this demo, it is the `./tests/` directory.


**Import the Local File System Test Provider from the `validmind.tests` module**


In [ ]:
from validmind.tests import LocalTestProvider

**Create an inline TestProvider Class that just returns a single test**


In [ ]:
import pandas as pd


class MySecondCustomTest(vm.vm_models.Metric):
    # The test name should match the content ID on the template
    name = "my_inline_provider.tests.MyCustomTest"

    def description(self):
        return "This is a custom test from an external test provider."

    def run(self):
        return self.cache_results([{"foo": "bar"}])

    def summary(self, results):
        return vm.vm_models.ResultSummary(
            results=[
                vm.vm_models.ResultTable(
                    data=pd.DataFrame(results),
                    metadata=vm.vm_models.ResultTableMetadata(
                        title="Results from Test Provider Inside Notebook"
                    ),
                )
            ]
        )


class TestProviderInline:
    def load_test(self, test_id):
        # ignore the test_id and just return the single test above
        return MySecondCustomTest

In [ ]:
# instantiate the test provider
inline_test_provider = TestProviderInline()
local_test_provider = LocalTestProvider(root_folder=".")

# register the test providers
vm.tests.register_test_provider(
    namespace="my_inline_provider",
    test_provider=inline_test_provider,
)  # validmind will now call the `TestProviderInline.load_test` method whenever it encounters a test ID that starts with `my_inline_provider`

vm.tests.register_test_provider(
    namespace="my_local_provider",
    test_provider=local_test_provider,
)  # validmind will now call the `LocalTestProvider.load_test` method whenever it encounters a test ID that starts with `my_local_provider`

<a id='toc4_3_'></a>

### Verify that `preview_template()` now loads the tests from the test providers

After registering the test providers with `vm.tests.register_test_provider()`, the developer framework can now locate the code that will execute the tests when we run the documentation tests on the template. We can verify this by running `preview_template()` again and seeing that the tests are now loaded correctly.


In [ ]:
vm.preview_template()

<a id='toc4_4_'></a>

### Running the template

Now we can run the template as usual and it will use the external test providers to load the appropriate tests. Note that we're not passing any inputs such as `dataset` and `model` to `run_documentation_tests()`. This is because our demo test providers do not have any required inputs and we're scoping the template execution to the `test_providers_demo` section by using the `section` keyword argument.


In [ ]:
suite_results = vm.run_documentation_tests(section="test_providers_demo")

<a id='toc5_'></a>

## Next steps

You can look at the output produced by the ValidMind Developer Framework right in the notebook where you ran the code, as you would expect. But there is a better way: use the ValidMind platform to work with your model documentation.

<a id='toc5_1_'></a>

### Work with your model documentation

1. From the [**Model Inventory**](https://app.prod.validmind.ai/model-inventory) in the ValidMind Platform UI, go to the model you registered earlier.

2. Click and expand the **Model Development** section.

3. Expand the **Test Providers Demo** section as defined in the template.

What you see is the full draft of your model documentation in a more easily consumable version. From here, you can make qualitative edits to model documentation, view guidelines, collaborate with validators, and submit your model documentation for approval when it's ready. [Learn more ...](https://docs.validmind.ai/guide/working-with-model-documentation.html)

<a id='toc5_2_'></a>

### Discover more learning resources

We offer many interactive notebooks to help you document models:

- [Run tests & test suites](https://docs.validmind.ai/guide/testing-overview.html)
- [Code samples](https://docs.validmind.ai/guide/samples-jupyter-notebooks.html)

Or, visit our [documentation](https://docs.validmind.ai/) to learn more about ValidMind.
